In [1]:
!pip install -q faiss-cpu faiss-gpu sentence-transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 25.8 MB/s eta 0:00:00


In [2]:
x = [1, 8, 3, 9, 1, 2, 9, 4, 5, 4, 6, 2]

In [7]:
m = 4
d = len(x)
assert d%m == 0
block_sz = int(d/m)
block_sz

3

In [8]:
u = [x[r:r+block_sz] for r in range(0, d, block_sz)]
u

[[1, 8, 3], [9, 1, 2], [9, 4, 5], [4, 6, 2]]

In [9]:
k = 2**5
assert k%m == 0
k_ = int(k/m)
k, k_

(32, 8)

In [16]:
from random import randint
c = []
for j in range(m):
  c_j = []
  for i in range(k_):
    c_ji = [randint(0, 9) for _ in range(block_sz)]
    c_j.append(c_ji)
  c.append(c_j)

In [17]:
def euclidean(v, u):
  dist = sum((x-y)**2 for x, y in zip(u, v))**.5
  return dist

def nearest(c_j, u_j):
  dist = 9e9
  for i in range(k_):
    new_dist = euclidean(c_j[i], u_j)
    if new_dist < dist:
      nearest_idx = i
      dist = new_dist
  return nearest_idx

In [18]:
ids = []
for j in range(m):
  ids.append(nearest(c[j], u[j]))
ids

[1, 2, 1, 3]

In [19]:
q = []
for j in range(m):
  cji = c[j][ids[j]]
  q.extend(cji)

In [20]:
q

[4, 9, 5, 6, 4, 4, 8, 4, 8, 2, 6, 1]

In [21]:
x

[1, 8, 3, 9, 1, 2, 9, 4, 5, 4, 6, 2]

In [22]:
import shutil
import urllib.request as request
from contextlib import closing

# first we download the Sift1M dataset
with closing(request.urlopen('ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz')) as r:
    with open('sift.tar.gz', 'wb') as f:
        shutil.copyfileobj(r, f)

In [23]:
import tarfile

# the download leaves us with a tar.gz file, we unzip it
tar = tarfile.open('sift.tar.gz', "r:gz")
tar.extractall()

In [24]:
import numpy as np

# now define a function to read the fvecs file format of Sift1M dataset
def read_fvecs(fp):
    a = np.fromfile(fp, dtype='int32')
    d = a[0]
    return a.reshape(-1, d + 1)[:, 1:].copy().view('float32')

In [25]:
# data we will search through
xb = read_fvecs('./sift/sift_base.fvecs')  # 1M samples
# also get some query vectors to search with
xq = read_fvecs('./sift/sift_query.fvecs')
# take just one query (there are many in sift_learn.fvecs)
xq = xq[0].reshape(1, xq.shape[1])

In [26]:
xb.shape, xq.shape

((1000000, 128), (1, 128))

In [68]:
import faiss
d = xb.shape[1]
m = 8
assert d%m == 0
nbits = 8
index = faiss.IndexPQ(d, m, nbits)
index.is_trained

False

In [69]:
%%time
index.train(xb)

CPU times: user 7.21 s, sys: 40.9 ms, total: 7.25 s
Wall time: 5.05 s


In [70]:
%%time
index.add(xb)

CPU times: user 9.48 s, sys: 6.13 s, total: 15.6 s
Wall time: 10.2 s


In [71]:
%%time
dist, I = index.search(xq, k=10)
I

CPU times: user 14.7 ms, sys: 0 ns, total: 14.7 ms
Wall time: 11.5 ms


array([[932085, 893601, 561813,  36267, 670103, 478814, 721370, 165581,
        872728, 934876]])

In [72]:
%%timeit
index.search(xq, k=10)

7.56 ms ± 83.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [73]:
l2_index = faiss.IndexFlatL2(d)
l2_index.is_trained

True

In [74]:
l2_index.add(xb)

In [75]:
l2_dist, l2_I = l2_index.search(xq, k=10)
l2_I

array([[932085, 934876, 561813, 708177, 706771, 695756, 435345, 701258,
        455537, 872728]])

In [76]:
recall = sum(1 for i in I[0] if i in l2_I)
recall

4

In [87]:
import os

def get_memory(index):
  faiss.write_index(index, './temp.index')
  msz = os.path.getsize('./temp.index')
  # os.remove('./temp.index')
  return msz

In [88]:
get_memory(l2_index)/get_memory(index)

62.967666475058046

In [92]:
m_sz = f"flat mem: {get_memory(l2_index)/1e6:.0f} MB"
print(m_sz)

flat mem: 512 MB


In [93]:
m_sz = f"pq mem: {get_memory(index)/1e6:.0f} MB"
print(m_sz)

pq mem: 8 MB


In [94]:
vecs = faiss.IndexFlatL2(d)
nlist = 2048
nbits = 8
index = faiss.IndexIVFPQ(vecs, d, nlist, m, nbits)
index.is_trained

False

In [95]:
%%time
index.train(xb)

CPU times: user 1min 41s, sys: 424 ms, total: 1min 41s
Wall time: 1min 28s


In [96]:
%%time
index.add(xb)

CPU times: user 30.4 s, sys: 6.98 s, total: 37.4 s
Wall time: 30.1 s


In [97]:
%%time
dist, idx = index.search(xq, k=10)

CPU times: user 254 µs, sys: 935 µs, total: 1.19 ms
Wall time: 6.16 ms


In [98]:
recall = sum(1 for i in idx[0] if i in l2_I)
recall

1

In [99]:
index.nprobe = 2048
dist, idx = index.search(xq, k=10)
sum(1 for i in idx[0] if i in l2_I)

3

In [100]:
index.nprobe = 2
dist, idx = index.search(xq, k=10)
sum(1 for i in idx[0] if i in l2_I)

2

In [101]:
index.nprobe = 48
dist, idx = index.search(xq, k=10)
sum(1 for i in idx[0] if i in l2_I)

3